# Train Sequential Neural Network

In [1]:
import re
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Lambda

import os
import numpy as np


# load in text 
raw_text = open(os.path.join(os.getcwd(), 'data_shakespeare/shakespeare.txt')).read()
lines = [line.split() for line in raw_text.split('\n\n') if line.split()]

# remove all unnecessary characters from the text
raw_text2 = ''
for line in lines:
    obs_elem = []
    for word in line:
        word = re.sub("\d+", "", word)
        if (word == ""):
            continue
        word = re.sub(r'[^-\w\']', '', word).lower()
        raw_text2 += word + ' '
            
# create same-length strings 
length = 40
# get list of all characters used in text
chars = sorted(list(set(raw_text2)))
# map characters to their numerical value
mapping = dict((c, i) for i, c in enumerate(chars))
vocab_size = len(mapping)

# tokenize a 40 length sequence and the character
# coming after it (the 40 length sequence will be 
# x and the 41st character will be y)
X = []
y = []
for i in range(length, int(len(raw_text2)/2)):
    # select sequence of tokens
    seq = raw_text2[i-length:i+1]
    # store
    encoded_seq = [mapping[c] for c in list(seq)]
    X.append(np.array(encoded_seq))
    output = raw_text2[i + 1]
    encoded_seq2 = mapping[output]
    y.append(encoded_seq2)
    
# separate into input and output
X = np.array(X)
y = np.array(y)
beforepX = X
# 
newsequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(newsequences)
y = to_categorical(y, num_classes=vocab_size)



Using TensorFlow backend.


# Generate poem from Sequential Neural Network

In [ ]:
def generate_poem(model, chars):
    start = np.random.randint(0, len(beforepX)-1)
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    pattern = list(beforepX[start])
    # generate characters
    poem = ''
    for k in range(1400):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = to_categorical(x, num_classes=vocab_size)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        poem = poem + result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    punctuation_list = [',', '.', ':', '?']
    punctuation_probs = [0.6, 0.1, 0.2, 0.1]
    poem_format = poem.split()
    for p in range(1, len(poem_format)):
        if ((p - 1) % 50 == 0):
            print(poem_format[p].capitalize() + ' ', end = '')
        else:
            print(poem_format[p], end = '')
            if (p % 50 == 0 and p < len(poem_format) - 1):
                print(np.random.choice(punctuation_list, p = punctuation_probs) + ' ', end = '')
                print()
            elif(p == len(poem_format) - 1):
                print('.', end = '')
            else:
                print(' ', end = '')
        
            
def summers_day_poem(model, chars):
    test = "shall i compare thee to a summers day sha"
    test = re.sub("\d+", " ", test)
    test = re.sub(r'[^-\w\']', ' ', test).lower()
    pattern = [mapping[c] for c in list(test)]
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    # generate characters
    poem = ''
    for k in range(1400):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = to_categorical(x, num_classes=vocab_size)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        poem = poem + result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    poem_format = poem.split()
    for p in range(1, len(poem_format)):
        if ((p - 1) % 50 == 0):
            print(poem_format[p].capitalize() + ' ', end = '')
        else:
            print(poem_format[p] + ' ', end = '')
        if (p % 50 == 0):
            print()

# Temperature = 1.5

In [ ]:
# define model
model1 = Sequential()
model1.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))

# adding temperature
temp = 1.5
model1.add(Lambda(lambda x : x /temp))
model1.add(Dense(vocab_size, activation='softmax'))

print(model1.summary())
# compile model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

model1.fit(X, y, epochs=90, verbose=2)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               184000    
_________________________________________________________________
lambda_1 (Lambda)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                5829      
Total params: 189,829
Trainable params: 189,829
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/90
 - 131s - loss: 2.4162 - acc: 0.3039
Epoch 2/90
 - 119s - loss: 2.0267 - acc: 0.4014
Epoch 3/90
 - 107s - loss: 1.8837 - acc: 0.4376
Epoch 4/90
 - 110s - loss: 1.7902 - acc: 0.4620
Epoch 5/90
 - 113s - loss: 1.7152 - acc: 0.4810
Epoch 6/90
 - 125s - loss: 1.6532 - acc: 0.4967
Epoch 7/90
 - 123s - loss: 1.5989 - acc: 0.5096
Epoch 8/90
 - 108s - loss: 1.5482 - acc: 0.5232
Epoch 9/90


In [ ]:
generate_poem(model1, chars)

In [ ]:
summers_day_poem(model1, chars)

# Temperature = 0.75

In [ ]:
model2 = Sequential()
model2.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))

# adding temperature
temp = 0.75
model2.add(Lambda(lambda x : x /temp))
model2.add(Dense(vocab_size, activation='softmax'))

print(model2.summary())
# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

model2.fit(X, y, epochs=90, verbose=2)

In [ ]:
generate_poem(model2, chars)

In [ ]:
summers_day_poem(model2, chars)

# Temperature = 0.25

In [ ]:
model3 = Sequential()
model3.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))

# adding temperature
temp = 0.25
model3.add(Lambda(lambda x : x /temp))
model3.add(Dense(vocab_size, activation='softmax'))

print(model3.summary())
# compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model

model3.fit(X, y, epochs=90, verbose=2)

In [ ]:
generate_poem(model3, chars)

In [ ]:
summers_day_poem(model3, chars)